# COMP90014 Assignment 1
### Semester 2, 2019

**\*\* This version of the assignment has corrected the guide answer in task 5** 

This assignment should be completed by each student individually. Make sure you read this entire document, and ask for help if anything is not clear. Any changes or clarification to this document will be announced via the LMS.

Please make sure you are aware of the University's rules on academic honesty and plagiarism, which are very strict: https://academichonesty.unimelb.edu.au/ 

Make sure you **do not** copy any code either from other students or from the internet. This is considered plagiarism. It is generally a good idea to avoid looking at any solutions as you may find it surprisingly difficult to generate your own solution to the problem once you have seen somebody else's.

Your completed notebook file containing all your answers will be turned in via LMS. No other files or formats will be accepted - only upload the completed `.ipynb` file.

### Overview
To complete the assignment you will need to finish the tasks in this notebook. There are multiple tasks that are connected in a logical order.

The tasks are a combination of writing your own implementations of algorithms we've discussed in lectures, writing your own code to use library implementations of these algorithms and interpreting the results in short answer format. Each short answer question has a word limit that will be strictly enforced!

In some case, we have provided test input and test output that you can use to try out your solutions. These tests are just samples and are not exhaustive - they may warn you if you've made a mistake, but they are not guaranteed to. It's up to you to decide whether your code is correct.

### Marking

Cells that must be completed to receive marks are clearly labeled. There are 16 graded cells, some of which are code cells, in which you must complete the code to solve a problem, and some of which are markdown cells, in which you must write your answers to short-answer questions. In addition to the graded cells, up to 8 marks will be given for code style, readability, efficiency and comments. 

The total marks for the assignment add up to 45, and it will be worth 15% of your overall subject grade.

### Background and data
During this assignment you will go through the steps to generate trees from a set of sequences. This includes calculating the distance matrices and the trees themselves using multiple algorithms. At each step of the process you will be asked to fill out short answer questions to demonstrate your understanding of the concepts.

The sequences you will use are the following [C2H2 zinc finger genes](https://en.wikipedia.org/wiki/Zinc_finger#Cys2His2) listed with links to their NCBI RefSeq records:
- [*KLF1*](https://www.ncbi.nlm.nih.gov/nuccore/NM_006563)
- [*KLF3*](https://www.ncbi.nlm.nih.gov/nuccore/NM_016531)
- [*KLF7*](https://www.ncbi.nlm.nih.gov/nuccore/NM_003709)
- [*YY1*](https://www.ncbi.nlm.nih.gov/nuccore/NM_003403)
- [*SP1*](https://www.ncbi.nlm.nih.gov/nuccore/NM_001251825)
- [*SP4*](https://www.ncbi.nlm.nih.gov/nuccore/NM_003112)
- [*SP8*](https://www.ncbi.nlm.nih.gov/nuccore/NM_182700)


### Task 1 - Setup

A new directory named `sequences`has been created in the same place as this notebook with a saved  FASTA file for each of the sequences above. The sequence files are named `<accession>.fa` where `<accession>` is the RefSeq accession number (e.g. `NM_006563.fa` for the *KLF1* sequence). Please look in the directory to undertsand the format of the data you will be working with. 

These files should show up when you `git pull`, you can run the code below to read in the sequences you just downloaded into a handy data structure. 

First, we create a dictionary `accessions` with gene names as keys and RefSeq accessions as values and populate it with the seven genes above.

Next we create a second dictionary `sequences` with gene names as keys and DNA sequence objects from the FASTA files as values. We read the FASTA files using the `scikit-bio` library method `skbio.sequence.DNA.read()`.

In [4]:
import os.path
import random
import skbio
from math import inf
from io import StringIO

In [5]:
accessions = dict()
sequences = dict()

accessions['KLF1'] = "NM_006563"
accessions['KLF3'] = "NM_016531"
accessions['KLF7'] = "NM_003709"
accessions['YY1'] = "NM_003403"
accessions['SP1'] = "NM_001251825"
accessions['SP4'] = "NM_003112"
accessions['SP8'] = "NM_182700"

for key in accessions.keys():
    with open(os.path.join("sequences", "{}.fa".format(accessions[key]))) as fasta:
        sequences[key] = skbio.sequence.DNA.read(fasta)      

Run the cell below to see an overview of the information we've just read in. Make sure you understand what is being printed, and ask for help if you don't.

In [6]:
for key in accessions.keys():
    print(key, accessions[key], len(sequences[key]), str(sequences[key])[:10], sep='\t')

KLF1	NM_006563	1645	TCAGAGTTCA
KLF3	NM_016531	5594	AGAGCGAGCG
KLF7	NM_003709	8365	AACAGCGCTG
YY1	NM_003403	6534	CTCCCTCTGC
SP1	NM_001251825	7536	ACCCCCCCCT
SP4	NM_003112	6077	ACAGCCCAGC
SP8	NM_182700	3626	ATTGTATTGC


## Part 1 - Distance calculations
Distances between sequences can be calculated in a number of different ways. In this part, you will calculate distance matrices for a set of sequences in two different ways. 

### Task 2 - Creating a distance matrix
Write a function `init_distance_matrix` that creates a distance matrix as a dictionary of dictionaries (similar to the substitution matrix in the Week 04 - Trees lab). The dictionary keys should be gene names and the values should be `None` after initialization.

In [7]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def init_distance_matrix(genes):
    """
    Take in a list of gene names and return a dict.
    The dict should be a nested 'dictionary of dictionaries', where the keys of the
    both the outer dict and the nested dicts are gene names.
    It should be possible to look up a value in the returned dict with a 
    call like dist[gene1][gene2].
    """
    dist = dict()

    ### YOUR CODE HERE - fill in the rest of this function

If you have implemented the code correctly, the following tests should give the right results:

In [40]:
# Should give: dict_keys(['KLF1', 'KLF3', 'KLF7', 'YY1', 'SP1', 'SP4', 'SP8'])
dist = init_distance_matrix(accessions.keys())
print(dist.keys())

dict_keys(['KLF1', 'KLF3', 'KLF7', 'YY1', 'SP1', 'SP4', 'SP8'])


In [41]:
# Should give: None
dist = init_distance_matrix(accessions.keys())
print(dist['KLF1']['SP1'])

None


In [42]:
# Should give an error!
dist = init_distance_matrix(accessions.keys())
print(dist['YY1']['SP4']['KLF1'])

TypeError: 'NoneType' object is not subscriptable

The following function will pretty-print a distance matrix that has been specified correctly.

In [26]:
def print_distance_matrix(dist):
    """
    Pretty-print a distance matrix.
    """
    genes = sorted(dist.keys())

    for s in [''] + genes:
        print("{: >10}".format(s), end='')
    print()

    for key1 in genes:
        print("{:10}".format(key1), end='')
        for key2 in genes:
            value = dist[key1][key2]
            if value is None:
                print("      None", end='')
            else:
                print("{: >#10.5f}".format(dist[key1][key2]), end='')
        print()

### Task 3 - Pairwise alignment scores as distances
The `skbio.alignment.local_pairwise_align_ssw()` library function calculates Striped Smith-Waterman alignments using a highly optimized implementation. It returns a tuple containing:
- `TabularMSA` object describing the alignment
- alignment score
- tuple of aligned position ranges

Here is some example output from calling `skbio.alignment.local_pairwise_align_ssw()`. Notice that changing the order of the sequences doesn't change the alignment.

In [27]:
skbio.alignment.local_pairwise_align_ssw(sequences['KLF1'], sequences['YY1'])

(TabularMSA[DNA]
 ----------------------------------
 Stats:
     sequence count: 2
     position count: 34
 ----------------------------------
 CTCCCAGATAAAAAAAAAAAAAAAAAAAAAAAAA
 CTCCCA---AAAAAAAAAAAAAAAAAAAAAAAAA, 53, [(1603, 1636), (4802, 4832)])

In [28]:
skbio.alignment.local_pairwise_align_ssw(sequences['YY1'], sequences['KLF1'])

(TabularMSA[DNA]
 ----------------------------------
 Stats:
     sequence count: 2
     position count: 34
 ----------------------------------
 CTCCCA---AAAAAAAAAAAAAAAAAAAAAAAAA
 CTCCCAGATAAAAAAAAAAAAAAAAAAAAAAAAA, 53, [(4802, 4832), (1603, 1636)])

In [29]:
# Get just the score, by indexing into the tuple
result_tuple = skbio.alignment.local_pairwise_align_ssw(sequences['KLF1'], sequences['YY1'])
result_tuple[1]

53

In [30]:
# Get just the tuples showing aligned ranges
skbio.alignment.local_pairwise_align_ssw(sequences['KLF1'], sequences['YY1'])[2]

[(1603, 1636), (4802, 4832)]

Complete the function `create_pairwise_dist_matrix` below. This function should create a  distance matrix using your function `init_distance_matrix`, then use `skbio.alignment.local_pairwise_align_ssw()` to populate it with alignment scores. `skbio.alignment.local_pairwise_align_ssw()` returns a tuple, and the second item in the tuple is the alignment score. Use `1 / score` as the value in the distance matrix.

Recall that this matrix will be symmetrical so your solution should only calculate each alignment once.

In [79]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def create_pairwise_dist_matrix(seqs):
    """
    Take in a dict with sequence names as keys and sequences as values.
    Return a dict representing the pairwise distances between all pairs of sequences.
    """
    ### YOUR CODE HERE - fill in this function

When you have implemented `create_pairwise_dist_matrix` correctly, the following should creat and print the filled-out distance matrix:

In [80]:
dist_pairwise = create_pairwise_dist_matrix(sequences)
print_distance_matrix(dist_pairwise)

dict_keys(['KLF1', 'KLF3', 'KLF7', 'YY1', 'SP1', 'SP4', 'SP8'])
                KLF1      KLF3      KLF7       SP1       SP4       SP8       YY1
KLF1         0.00000   0.01220   0.00833   0.02174   0.02500   0.00962   0.01887
KLF3         0.01220   0.00000   0.00855   0.03125   0.02439   0.02941   0.03448
KLF7         0.00833   0.00855   0.00000   0.02222   0.02632   0.02857   0.02326
SP1          0.02174   0.03125   0.02222   0.00000   0.00446   0.00606   0.02564
SP4          0.02500   0.02439   0.02632   0.00446   0.00000   0.01562   0.02326
SP8          0.00962   0.02941   0.02857   0.00606   0.01562   0.00000   0.01852
YY1          0.01887   0.03448   0.02326   0.02564   0.02326   0.01852   0.00000


Test that your function produces the correct results on the toy sequences below. The below code should print

                    GeneA     GeneB     GeneC
    GeneA        0.00000   0.10000   0.25000
    GeneB        0.10000   0.00000   0.25000
    GeneC        0.25000   0.25000   0.00000


In [81]:
test_sequences = {'GeneA': skbio.DNA('GATTACA'), 'GeneB': skbio.DNA('GTTACAT'), 'GeneC':skbio.DNA('AGCTATG')}
test_dist_pairwise = create_pairwise_dist_matrix(test_sequences)
print_distance_matrix(test_dist_pairwise)

dict_keys(['GeneA', 'GeneB', 'GeneC'])
               GeneA     GeneB     GeneC
GeneA        0.00000   0.10000   0.25000
GeneB        0.10000   0.00000   0.25000
GeneC        0.25000   0.25000   0.00000


#### Task 3 - Question 1
Why do we use `1 / score` instead of `score` for the distance? (max 50 words).

Write your answer in the markdown cell below.


~~ GRADED CELL (3 marks) - complete this cell ~~

*Your answer here*

### Task 4 - Multiple alignment distances
The following code will load the precomputed protein multiple sequence alignment of our genes (generated using Clustal $\Omega$) and read it into a `TabularNSA` object. Download the `znfs.clustal_num` file from the LMS and put it in the same directory as the notebook.

In [82]:
with open("znfs.clustal_num") as znfs:
    znf_msa = skbio.TabularMSA.read(znfs, constructor=skbio.Protein)
znf_msa

TabularMSA[Protein]
-----------------------------------------------------------------------
Stats:
    sequence count: 7
    position count: 998
-----------------------------------------------------------------------
--------------------------------- ... ---------------------------------
--------------------------------- ... ---------------------------------
...
MSDQDHSMDEMTAVVKI---------------- ... AICPEGIARLANSGINVMQVADLQSINISGNGF
MSDQKKEEEEEAAAAAAMATEGGKTSEPENNNK ... AISQDSNPATPNVSTNMEE-------------F

Now, you will write some code to calculate pairwise identity from the multiple alignment. The `TabularMSA` is indexed by gene name. We can extract a subset with a pair of genes by indexing with a list of two gene names, using the following syntax, which will be familiar to students that have used `pandas` for data science in Python:

In [83]:
znf_msa.loc[['KLF1', 'YY1']]

TabularMSA[Protein]
-----------------------------------------------------------------------
Stats:
    sequence count: 2
    position count: 998
-----------------------------------------------------------------------
--------------------------------- ... ---------------------------------
--------------------------------- ... ---------------------------------

The following example code iterates over all positions using the `.iter_positons()` method. This returns a `Sequence` object for each column. It skips positions where either sequence contains a gap and counts matches and mismatches. The number of matches and mismatches can be used to calculate the percent identity between two aligned sequences (# matches/ # matches + # mismatches).

In [84]:
match = 0
mismatch = 0
for p1, p2 in znf_msa.loc[['KLF1', 'YY1']].iter_positions():
    if str(p1) == '-' or str(p2) == '-':
        continue
    elif str(p1) == str(p2):
        match += 1
    else:
        mismatch += 1
print("Matches: {}\nMismatches: {}".format(match, mismatch))

Matches: 66
Mismatches: 184


Complete the function `create_msa_dist_matrix` below. This function should start by creating a new distance matrix using your `init_distance_matrix` function from **Task 1**, then fill each cell of the distance matrix with the fraction of non-gap-containing columns with mismatches (or one minus the percent identity) for each pair of sequences in the multiple alignment. As a guide, try generalising the code above.

In [95]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def create_msa_dist_matrix(seqs, msa):
    """
    Take in a dict sequences where keys are sequence names and values are sequences, 
    and a multiple sequence alignment object znf_msa.
    Return a dict representing a distance matrix, where the values in the distance
    matrix represent the percent identity between the sequences - i.e. the fraction
    of mismatches out of all matches and mismatches.
    """
    ### YOUR CODE HERE - fill in this function


When you have implemented `create_msa_dist_matrix` correctly, the following will create and print the filled-out percent-identity matrix:

In [90]:
dist_msa = create_msa_dist_matrix(sequences, znf_msa)
print_distance_matrix(dist_msa)

                KLF1      KLF3      KLF7       SP1       SP4       SP8       YY1
KLF1         0.00000   0.64706   0.64632   0.67232   0.67600   0.67160   0.67410
KLF3         0.66775   0.00000   0.66024   0.67364   0.68040   0.68063   0.66862
KLF7         0.67784   0.67269   0.00000   0.67847   0.67876   0.67730   0.67913
SP1          0.69877   0.69964   0.69859   0.00000   0.69113   0.68936   0.70207
SP4          0.68941   0.69116   0.69098   0.68556   0.00000   0.68401   0.69353
SP8          0.68320   0.68321   0.68255   0.68374   0.68248   0.00000   0.68474
YY1          0.68038   0.68332   0.68763   0.69274   0.69673   0.70010   0.00000


Test that your function produces the correct results on the toy sequences below. The below code should print

                   GeneA     GeneB     GeneC
    GeneA        0.00000   0.16667   0.71429
    GeneB        0.16667   0.00000   0.50000
    GeneC        0.71429   0.50000   0.00000

In [96]:
test_sequences = {'GeneA': skbio.DNA('GATTACA'), 'GeneB': skbio.DNA('GTTACAT'), 'GeneC':skbio.DNA('AGCTATG')}
test_multiple_alignment_output = """CLUSTAL O(1.2.4) multiple sequence alignment


GeneA      GATTACA-	7
GeneB      -GTTACAT	7
GeneC      AGCTATG-	7"""
test_msa = skbio.TabularMSA.read(StringIO(test_multiple_alignment_output), constructor=skbio.DNA)
test_msa_dist = create_msa_dist_matrix(test_sequences, test_msa)
print_distance_matrix(test_msa_dist)

               GeneA     GeneB     GeneC
GeneA        0.00000   0.83333   0.53846
GeneB        0.63158   0.00000   0.60000
GeneC        0.53125   0.52632   0.00000


#### Task 4 - Question 1
Do you expect the distance matrices for MSA and SSW to be similar? Why or why not? (max 50 words)

Write your answer in the markdown cell below.

~~ GRADED CELL (3 marks) - complete this cell ~~

*Your answer here*

### Task 5 - k-mer distances
The k-mer distance between two sequences is the fraction of k-mers that are unique to either sequence. Write a function that calculates a dictionary of k-mers (for `k` = any number) and their counts for each gene sequence. We will not use the counts to calculate the distance but they are useful for other applications.

Be sure to test your function with a short example sequence you can work by hand.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def count_kmers(seq, k):
    """
    Take in a sequence and a value k.
    Return all kmers of length k in the form of a dict
    with k-mer strings as keys and counts as values.
    """
    ### YOUR CODE HERE - fill in this function


When you have implemented `count_kmers` correctly, the following tests should give the right results:

In [ ]:
# Should give: {'GTA': 2, 'TAA': 1, 'AAG': 1, 'AGT': 1}
count_kmers('GTAAGTA', 3)

In [ ]:
# Should give: {'GTA': 2, 'TAG': 1, 'AGT': 1, 'TAA': 1}
count_kmers('GTAGTAA', 3)

In [ ]:
# Should give: {'GT': 2, 'TA': 2, 'AG': 1, 'AA': 1}
count_kmers('GTAGTAA', 2)

Write a function `count_kmers_for_sequences` that uses your previous function to count k-mers for all sequences in a dictionary like `sequences`. It should return a new dictionary with gene names as keys and k-mer dictionaries as values.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def count_kmers_for_sequences(seqs, k):
    """
    Given a dict of sequences, build a dict of the kmer counts for each
    sequence. Keys should be sequence names, and values should be kmer count
    dicts (where keys are kmers and values are counts).
    """
    ### YOUR CODE HERE - fill in this function


The below code creates the kmer_counts dict, storing all kmer counts for each sequence. 

It then prints the name of each gene, the number of unique k-mers in each gene and the gene length, one gene per line separated by a tab. If your implementation of `count_kmers_for_sequences` is correct, it should produce the output

**These numbers have been corrected** 
    
    KLF1	1111	1645
    KLF3	2633	5594
    KLF7	3044	8365
    YY1	 2858	6534
    SP1	 2756	7536
    SP4	 2522	6077
    SP8	 2012	3626


In [ ]:
kmer_counts = count_kmers_for_sequences(sequences, k=6)

for key in kmer_counts.keys():
    print(key, len(kmer_counts[key]), len(sequences[key]), sep='\t')

#### Task 5 - Question 1
How well correlated are the unique k-mer counts and the gene lengths? What does this suggest about the sequences? (max 50 words)

Write your answer in the markdown cell below.

~~ GRADED CELL (3 marks) - complete this cell ~~

*Your answer here*

Now write a function to calculate the k-mer distance between a pair of genes given their k-mer count dictionaries. Recall that the k-mer distance is the fraction of k-mers that are unique to one of the sequences. You may find the Python `set` structure useful.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def kmer_distance(k1, k2):
    """
    Given two k-mer count dictionaries, calculate the k-mer distance between these two sets of kmers.
    The counts themselves are not used. Return a floating-point number for the distance.
    """
    ### YOUR CODE HERE - fill in this function


When you have implemented the `kmer_distance` function correctly, this test should give the right result:

In [ ]:
# Should give: 0.4
test_counts_1 = {'GTA': 2, 'TAA': 1, 'AAG': 1, 'AGT': 1} 
test_counts_2 = {'GTA': 2, 'TAG': 1, 'AGT': 1, 'TAA': 1} 
kmer_distance(test_counts_1, test_counts_2)

Complete the function `create_kmer_dist_matrix` below. This function should create a new distance matrix containining k-mer distances. The matrix, as before, should be returned as a dict of dicts. Use your `kmer_distance` function and your `init_distance_matrix` function.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def create_kmer_dist_matrix(counts):
    """
    Take in a kmer counts dict, where keys are gene names and values are
    dicts of kmer counts (i.e. kmers as keys and counts as values). 
    Return a kmer distance matrix in the form of a dict of dicts of distances.
    """
    ### YOUR CODE HERE - fill in this function


In [ ]:
dist_kmer = create_kmer_dist_matrix(kmer_counts)
print_distance_matrix(dist_kmer)

If you've implemented `create_kmer_dist_matrix` correctly, the following test case should display

                   GeneA     GeneB
    GeneA        0.00000   0.40000
    GeneB        0.40000   0.00000

In [ ]:
test_counts = {'GeneA': {'GTA': 2, 'TAA': 1, 'AAG': 1, 'AGT': 1},
               'GeneB': {'GTA': 2, 'TAG': 1, 'AGT': 1, 'TAA': 1}} 
test_kmer_dist_matrix = create_kmer_dist_matrix(test_counts)
print_distance_matrix(test_kmer_dist_matrix)

Check your implementation by completing the `create_kmer_dist_matrix_skbio` function below. It is very similar to the function above, but in this case you build a distance matrix by calculating k-mer distance using `skbio.sequence.distance.kmer_distance()`. The library function takes as input two sequences (from your `sequences` dictionary) and a value for `k`.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def create_kmer_dist_matrix_skbio(seqs):
    """
    Take in a kmer counts dict, where keys are gene names and values are
    dicts of kmer counts (i.e. kmers as keys and counts as values). 
    Return a kmer distance matrix in the form of a dict of dicts of distances.
    
    Use skbio.sequence.distance.kmer_distance() to calculate distances for each cell.
    """
    ### YOUR CODE HERE - fill in this function


In [ ]:
dist_kmer_lib = create_kmer_dist_matrix_skbio(sequences)
print_distance_matrix(dist_kmer_lib)

### Task 6 - Building trees with UPGMA
In this section you will implement the UPGMA algorithm to create ultrameric trees as described in lectures. You will output the trees in Newick format and import those Newick strings into an `skbio` library function to draw the trees as ASCII art.

To implement UPGMA, you should split the process into computable steps, write a function for each step and then use the functions to build your tree.

The first step of the UPGMA algorithm is finding the minimum distance in the matrix. Write a function `minimum_distance` that takes a distance matrix and returns a tuple with the distance and the two node names corresponding to the minimum distance.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def minimum_distance(dist):
    """
    Find and return a tuple containing the minimum distance
    in the distance matrix and the nodes names corresponding 
    to the minimum.
    """
    ### YOUR CODE HERE - fill in this function


When you have implemented `minimum_distance` correctly, the following tests should give the right result:

In [ ]:
# Should give: (3.3, 'geneB', 'geneC')
test_dist = {'geneA': {'geneA':0, 'geneB':5, 'geneC':12},
             'geneB': {'geneA':5, 'geneB':0, 'geneC':3.3},
             'geneC': {'geneA':12, 'geneB':3.3, 'geneC':0}}
print(minimum_distance(test_dist))

In [ ]:
# Should give: (4.1, 'geneA', 'geneB')
test_dist = {'geneA': {'geneA':0, 'geneB':4.1, 'geneC':5},
             'geneB': {'geneA':4.1, 'geneB':0, 'geneC':7.2},
             'geneC': {'geneA':5, 'geneB':7.2, 'geneC':0}}
print(minimum_distance(test_dist))

Write another function `update_distances` that takes a distance matrix, the names of the nodes being combined, the name of the newly created node and a dictionary of node sizes and returns a new distance matrix with the result of merging and creating the new node.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def update_distances(dist, a, b, new_node, node_sizes):
    """
    Take in:
      dist : a distance matrix, structured as a dict of dicts of distances,
             with node names as keys
      a, b : names of nodes to merge
      new_node : the name of the new node to create 
      node_sizes : a dict giving the weightings (cluster sizes) of existing nodes,
                   with names as keys and sizes as values
    Return the new distance matrix, which will contain a node new_node
    and will not contain a or b.
    """
    ### YOUR CODE HERE - fill in this function


Here are some test cases you can use in testing `update_distances`:

The below cell should print 

                   geneC    nodeAB
    geneC        0.00000   6.10000
    nodeAB       6.10000   0.00000

In [ ]:
# Should return {'geneC': {'geneC': 0, 'nodeAB': 6.1}, 'nodeAB': {'geneC': 6.1, 'nodeAB': 0.0}}, 
# and also print the test output as shown above
test_dist = {'geneA': {'geneA':0, 'geneB':4.1, 'geneC':5},
             'geneB': {'geneA':4.1, 'geneB':0, 'geneC':7.2},
             'geneC': {'geneA':5, 'geneB':7.2, 'geneC':0}}
test_sizes = {'geneA': 1, 'geneB': 1, 'geneC': 1}
test_output = update_distances(test_dist, 'geneA', 'geneB', 'nodeAB', test_sizes)
print(test_output)
print_distance_matrix(test_output)

The below cell should print 

                   geneC    nodeAB
    geneC        0.00000   5.55000
    nodeAB       5.55000   0.00000

In [ ]:
# Should return {'geneC': {'geneC': 0, 'nodeAB': 5.55}, 'nodeAB': {'geneC': 5.55, 'nodeAB': 0.0}}, 
# and also print the test output as shown above
test_dist = {'nodeA': {'nodeA':0, 'geneB':4.1, 'geneC':5},
             'geneB': {'nodeA':4.1, 'geneB':0, 'geneC':7.2},
             'geneC': {'nodeA':5, 'geneB':7.2, 'geneC':0}}
test_sizes = {'nodeA': 3, 'geneB': 1, 'geneC': 1}
test_output = update_distances(test_dist, 'nodeA', 'geneB', 'nodeAB', test_sizes)
print(test_output)
print_distance_matrix(test_output)

Write a final function `upgma` that takes a distance matrix as input and returns a tree as a Newick formatted string.

Because we want to output our tree in Newick format, you will need an additional dictionary of Newick strings for each node that gets created. You also need to keep track of the branch lengths. Recall that the $D_{ij}$ in the algorithm from lecture is the length to the tip not the internal branch length, but the Newick string expects the internal branch lengths.

In [ ]:
# ~~ GRADED CELL (2 marks) - complete this cell ~~

def upgma(dist):
    """
    Build a UPGMA tree based on the given distance matrix
    and return the tree as a string in Newick format.
    """
    ### YOUR CODE HERE - fill in this function


Here is a test case you can use to help you test your `upgma` solution:

In [ ]:
# Should give: '(geneC:3.05,(geneA:2.05,geneB:2.05):1.0);'
test_dist = {'geneA': {'geneA':0, 'geneB':4.1, 'geneC':5},
             'geneB': {'geneA':4.1, 'geneB':0, 'geneC':7.2},
             'geneC': {'geneA':5, 'geneB':7.2, 'geneC':0}}
upgma(test_dist)

Let's try UPGMA out using different distance measures. The following three blocks generate trees using your `upgma` function and print those trees to the console. There is one block per distance method.

In [ ]:
newick = upgma(dist_pairwise)
t = skbio.io.read(StringIO(newick), format="newick", into=skbio.tree.TreeNode)
print("Local alignment score distance, UPGMA")
print()
print(newick)
print()
print(t.ascii_art())

In [ ]:
newick = upgma(dist_msa)
t = skbio.io.read(StringIO(newick), format="newick", into=skbio.tree.TreeNode)
print("Multiple sequence alignment percent identity distance, UPGMA")
print()
print(t)
print()
print(t.ascii_art())

In [ ]:
newick = upgma(dist_kmer)
t = skbio.io.read(StringIO(newick), format="newick", into=skbio.tree.TreeNode)
print("K-mer distance, UPGMA")
print()
print(t)
print()
print(t.ascii_art())

#### Task 6 - Question 1
Why are the trees different? (max 50 words)

Write your answer in the markdown cell below.

~~ GRADED CELL (3 marks) - complete this cell ~~

*Your answer here*

### Task 7 - Building trees with neighbor joining
In this section you will use the `skbio` implementation of the neighbor joining algorithm to build trees and visualize them as in the previous section.

The following code converts a dictionary-based distance matrix to a `DistanceMatrix` object suitable for the `skbio` neighbor joining implementation.

In [4]:
def convert_distance_matrix(dist):
    genes = sorted(dist.keys())
    dist_lists = list()

    for key1 in genes:
        a = list()
        for key2 in genes:
            a.append(dist[key1][key2])
        dist_lists.append(a)
        
    return skbio.DistanceMatrix(dist_lists, genes)

The following three blocks generate trees using the `skbio` neighbor joining implementation and print those trees to the console. There is one block per distance method.

In [5]:
m = convert_distance_matrix(dist_pairwise)
t = skbio.tree.nj(m)
print("Local alignment score distance, NJ")
print()
print(t)
print()
print(t.ascii_art())

NameError: name 'dist_pairwise' is not defined

In [ ]:
m = convert_distance_matrix(dist_msa)
t = skbio.tree.nj(m)
print("Multiple sequence alignment percent identity distance, NJ")
print()
print(t)
print()
print(t.ascii_art())

In [ ]:
m = convert_distance_matrix(dist_kmer)
t = skbio.tree.nj(m)
print("K-mer distance, NJ")
print()
print(t)
print()
print(t.ascii_art())

#### Task 7 - Question 1
Based on all the trees you've generated, which distance method do you think is the best and why? (max 100 words)

Write your answer in the markdown cell below.

~~ GRADED CELL (3 marks) - complete this cell ~~

*Your answer here*